# Library Nodes

`LibraryNode`s facilitate the abstraction of common operations, enabling easy reuse in different SDFGs and Data-Centric progrmas. This tutorial covers creating `LibraryNode`s with different implelementations (called *expansions* or `ExpandTransformation`s), and how to use them in SDFGs or Data-Centric programs.

For this tutorial, we use as an example the SDDMM (sampled dense-dense matrix multiplication) operation:
$$\bm{D} = \bm{A} \odot \left(\bm{B} \times \bm{C}\right)$$
$\bm{A}$ is a sparse matrix, while $\bm{B}$ and $\bm{C}$ are dense matrices. The ouput $\bm{D}$ is the Hadamard (element-wise) product of $\bm{A}$ and the matrix product of $\bm{B}$ and $\bm{C}$, and has the same sparsity pattern as $\bm{A}$. Effectively, $\bm{A}$ *samples* (or filters) the dense product $\bm{B} \times \bm{C}$. Assuming $\bm{A}$ is in CSR format, the SDDMM algorithm is as follows:

```python
# A (D) has shape (M, N) with nnz non-zero values
# A_data (D_data) is the non-zero values of A (D)
# A_indices (D_indices) is the column indices of A (D)
# A_indptr (D_indptr) is the row pointers of A (D)
# B has shape (M, K)
# C has shape (K, N)
D_data = np.ones_like(A_data)
D_indices = np.copy(A_indices)
D_indptr = np.copy(A_indptr)
for i in range(M):
    for j in range(A_indptr[i], A_indptr[i + 1]):
        for k in range(K):
            D_data[j] *= B[i, k] * C[k, A_indices[j]]
```

We start by creating a LibraryNode that represents the SDDMM operation. We create a class that inherits from `dace.sdfg.nodes.LibraryNode`, and we decorate it with `@dace.library.node`. The class must include an `implementations` dictionary, and an `defaul_implementation` string, which we will discuss later. The `LibraryNode`'s initialization method must call the initialization method of the super-class and pass the node's name, location, inputs, and outputs. The inputs and the outputs are the node's connector names.

In [7]:
import dace

from dace import library
from dace.sdfg import nodes
from dace.transformation import ExpandTransformation
from typing import Dict


@library.node
class MySDDMM(nodes.LibraryNode):

    # We will fill those later
    implementations: Dict[str, ExpandTransformation] = {}
    default_implementation: str = None

    def __init__(self, name, location=None):
        super().__init__(name,
                         location=location,
                         inputs={'_a_data', '_a_indices', '_a_indptr', '_b', '_c'},
                         outputs={'_d_data', '_d_indices', '_d_indptr'})


A `LibraryNode` can have different implemenetations (expansions), generic or specialized for specific architectures. These implementations can use the SDFG API but they can also be written as Data-Centric programs. We start by creating a *pure* expansion, which is an implementation that does not use any components, e.g., libraries, external to DaCe. We write this expansion as a Data-Centric Python program:

In [8]:
@library.expansion
class MySDDMMPureExpansion(ExpandTransformation):

    environments = []

    @staticmethod
    def expansion(node, state, sdfg):

        # Find shapes and datatypes of inputs and outputs

        # A matrix
        a_indptr_name = list(state.in_edges_by_connector(node, '_a_indptr'))[0].data.data
        a_indptr_arr = sdfg.arrays[a_indptr_name]
        a_data_name = list(state.in_edges_by_connector(node, '_a_data'))[0].data.data
        a_data_arr = sdfg.arrays[a_data_name]
        a_rowsp1 = a_indptr_arr.shape[0]
        a_nnz = a_data_arr.shape[0]
        a_dtype = a_data_arr.dtype

        # B matrix
        b_name = list(state.in_edges_by_connector(node, '_b'))[0].data.data
        b_arr = sdfg.arrays[b_name]
        b_rows = b_arr.shape[0]
        b_cols = b_arr.shape[1]
        b_dtype = b_arr.dtype

        # C matrix
        c_name = list(state.in_edges_by_connector(node, '_c'))[0].data.data
        c_arr = sdfg.arrays[c_name]
        c_rows = c_arr.shape[0]
        c_cols = c_arr.shape[1]
        c_dtype = c_arr.dtype

        # D matrix
        # We assume that it has the same shape and datatype as A

        @dace.program
        def sddmm_pure(_a_data: a_dtype[a_nnz], _a_indices: dace.int32[a_nnz], _a_indptr: dace.int32[a_rowsp1],
                       _b: b_dtype[b_rows, b_cols], _c: c_dtype[c_rows, c_cols],
                       _d_data: a_dtype[a_nnz], _d_indices: dace.int32[a_nnz], _d_indptr: dace.int32[a_rowsp1]):

            _d_data[:] = 1
            _d_indices[:] = _a_indices
            _d_indptr[:] = _a_indptr

            for i in dace.map[0:a_rowsp1 - 1]:
                for j in dace.map[_a_indptr[i]:_a_indptr[i + 1]]:
                    for k in dace.map[0:b_cols]:
                        _d_data[j] *= _b[i, k] * _c[k, _a_indices[j]]

        return sddmm_pure.to_sdfg()

